## Importing the llm (OpenAI)

In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI

load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
llm = OpenAI(openai_api_key=OPENAI_API_KEY,temperature=0)

text = "What would be a good company name for a company that makes AI tools & products for businesses?"
print(llm(text))



SmartTech Solutions


## Loading pdf and splitting to chunks

In [2]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./IndianConstitution.pdf")
pages = loader.load_and_split()

print(len(pages))

403


In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

texts = text_splitter.split_documents(pages)

print(len(texts))

403


## Embedding texts and then storing in vector database

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
import pinecone

PINECONE_API_KEY=os.getenv('PINECONE_API_KEY')
PINECONE_ENV=os.getenv('PINECONE_ENV')

embeddings = OpenAIEmbeddings()

# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

index_name = "kritrim-ai"

docsearch = Pinecone.from_documents(texts, embeddings, index_name=index_name)

/home/deependu/.local/lib/python3.10/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Now we will perform similarity search to return the most similar pieces from the documents

In [5]:
query = "What are the laws regarding the right to privacy in India?"
docs = docsearch.similarity_search(query)

print(docs)

[Document(page_content='THE CONSTITUTION OF  INDIA \n(Part III.—Fundamental Rights) 14\nRight against Exploitation \n23. Prohibition of traffic in human beings and forced labour. —(1) \nTraffic in human beings and begar and other similar forms of forced labour are \nprohibited and any contravention of this provision shall be an offence \npunishable in accordance with law. \n(2) Nothing in this article shall prevent the State from imposing \ncompulsory service for public purposes, and in imposing such service the State \nshall not make any discrimination on grounds only of religion, race, caste or \nclass or any of them. \n24. Prohibition of employment of children in factories, etc. —No child \nbelow the age of fourteen years shall be employed to work in any factory or \nmine or engaged in any other hazardous employment. \nRight to Freedom of Religion \n25. Freedom of conscience and free profession, practice and \npropagation of religion. —(1) Subject to public order, morality and healt

## Now LLM will generate a summary of the most similar pieces

In [6]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever()
)

print(qa.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


## Modifying prompt

In [10]:
new_prompt = '''Use the following pieces of context to answer the question at the end. If the answer is not present in the context, please answer with "I am sorry, but I couldn't find the answer".

{context}

Question: {question}
Helpful Answer:'''

qa.combine_documents_chain.llm_chain.prompt.template = new_prompt

In [11]:
res = qa.run("What are the laws regarding buying property in Kashmir?")
print(res)

 The executive power of the Union and of each State shall extend to the acquisition, holding and disposal of property and the making of contracts for any purpose. However, the said executive power of the Union shall, in so far as such trade or business or such purpose is not one with respect to which Parliament may make laws, be subject in each State to legislation by the State.


In [13]:
res = qa.run("What are the conditions for being considered an Indian?")
print(res)

 According to the Constitution of India, every person who has his domicile in the territory of India and meets any of the following conditions is considered a citizen of India: 
(a) born in the territory of India; 
(b) either of whose parents was born in the territory of India; 
(c) has been ordinarily resident in the territory of India for not less than five years immediately preceding such commencement; 
(d) migrated to the territory of India from the territory now included in Pakistan and meets the requirements of Article 6; or 
(e) is of Indian origin and is registered as a citizen of India by the diplomatic or consular representative of India in the country where the person is for the time being residing.
